In [32]:
import pandas as pd
import os
import uuid
import requests
import requests_cache
from datetime import timedelta
from typing import Union

session = requests_cache.CachedSession('paper_cache', expire_after=timedelta(hours=24))

# Read in authors

In [33]:
filepath = os.path.join('data', 'author_list.xlsx')
authors = pd.read_excel(filepath)

In [34]:
authors.columns

Index(['Index', 'Last Name', 'First Name', 'Orcid', 'google_scholar', 'pubmed',
       'institution_url'],
      dtype='object')

## Remove authors with a duplicate ORCID

In [35]:
deduped = authors[~authors.duplicated(subset='Orcid') | authors['Orcid'].isna() ]

## Remove authors with a duplicate ORCID/no ORCID keeping ORCID

In [36]:
name = deduped['Last Name'].str.strip().str.lower() + deduped['First Name'].str.strip().str.lower().str.get(0)
deduped['name'] = name.copy()
deduped = deduped[~name.duplicated(keep='first')].sort_values('Last Name')

/var/folders/_7/bccbfn110b501smf76mty7xh0000gn/T/ipykernel_41279/4072643918.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  deduped['name'] = name.copy()


In [37]:
deduped

,Index,Last Name,First Name,Orcid,google_scholar,pubmed,institution_url,name
54,59,Abraham,Edo,https://orcid.org/0000-0003-0989-5456,NaN,NaN,NaN,abrahame
249,60,Adam,Jean-Paul,NaN,NaN,NaN,NaN,adamj
162,31,Agarwala,Matthew,https://orcid.org/0000-0002-0042-2559,NaN,NaN,NaN,agarwalam
250,61,Agbemabiese,Lawrence,NaN,https://scholar.google.co.uk/citations?hl=en&u...,NaN,NaN,agbemabiesel
14,62,Agutu,Churchill,https://orcid.org/0000-0003-3511-9960,NaN,NaN,NaN,agutuc
...,...,...,...,...,...,...,...,...
174,242,Yeganyan,Rudolf,https://orcid.org/0000-0001-9325-8147,NaN,NaN,NaN,yeganyanr
72,104,Yussuff,Abdulmutalib,https://orcid.org/0000-0002-9565-305X,NaN,NaN,NaN,yussuffa
343,116,Zhou,Zhaoqi,NaN,NaN,NaN,https://www.geog.ox.ac.uk/graduate/research/zz...,zhouz
263,150,d'Avezac,Mayeul,NaN,NaN,NaN,NaN,d'avezacm


In [38]:
deduped['uuid'] = deduped['Last Name'].apply(lambda x: str(uuid.uuid4()))

# Read in papers and relations

In [66]:
filepath = os.path.join('data', 'papers.xlsx')
papers = pd.read_excel(filepath, sheet_name='Papers')
relations = pd.read_excel(filepath, sheet_name='Author Paper')

papers['paper_uuid'] = papers['DOI'].apply(lambda x: str(uuid.uuid4()))

relations = relations.merge(papers[['DOI', 'paper_uuid']], left_on='DOI', right_on='DOI', how='left')

# papers['DOI'] = 'https://doi.org/' + papers['DOI']

In [67]:
# Read in modified authors file
authors = pd.read_csv('data/authors_master.csv')
authors['name'] = authors['Last Name'].str.strip().str.lower() + authors['First Name'].str.strip().str.lower().str.get(0)
authors['uuid'] = authors['Last Name'].apply(lambda x: str(uuid.uuid4()))

In [68]:
authors_surname = authors.set_index('name')

relations['join_name'] = relations['Last_Name'].str.strip().str.lower() + relations['First_Name'].str.strip().str.lower().str.get(0)

joined = relations.join(other=authors_surname, on='join_name', validate="m:m")

In [83]:
papers

,DOI,Abstract,paper_uuid
0,10.1038/s41560-022-01152-0,Aligning development and climate goals means A...,bdcd53b3-034c-479d-8115-aaaf1d520f2c
1,10.3390/en14185827,Almost all countries have committed to develop...,9f8598d1-7fa9-4056-80e3-42eb7608eeda
2,10.1038/s41598-023-28377-7,"As the world transitions to net zero, energy s...",4f168d4a-78fd-4c92-8597-6da222b17642
3,10.1016/j.respol.2022.104528,Business model innovation (BMI) is often compl...,ba2a928d-69c2-48ef-a077-e30ebea64491
4,10.1016/j.esr.2022.100890,Deploying low-carbon electricity systems in de...,ddf2cfd5-cf3b-4d8e-92df-9c905cce1b49
5,10.1038/s41560-022-01121-7,Energy development in Indigenous lands has bee...,e19e1b94-105f-489c-9380-98fc395c4ffa
6,10.1016/j.esr.2021.100650,Energy modelling is critical for addressing ch...,6425b47b-65de-4bb2-9491-21a403b58efd
7,10.3390/en14041209,"Ethiopia is a low-income country, with low ele...",f012c5b9-78b3-4713-ac70-8c4454788022
8,10.1016/j.esr.2021.100799,Financial institutions around the world have a...,2507dd90-e6b6-4c8e-a8b4-99d49629739c
9,10.1016/j.jclepro.2022.131014,Formally adopting climate change mitigation po...,23b848e4-745b-4aa7-baff-72866d673c02


In [72]:

# joined = joined.set_index('DOI').join(other=papers.set_index('DOI'), on='DOI', validate="m:1")
joined

,First_Name,Last_Name,DOI,paper_uuid,join_name,First Name,Last Name,Orcid,google_scholar,pubmed,institution_url,gender,uuid
0,Liu,Yang,10.1007/978-3-031-20179-0_3,6d82605a-4b5d-4c5a-9360-6dd8ee3a6051,yangl,Liu,Yang,https://orcid.org/0000-0002-3363-8620,NaN,NaN,NaN,male,65fd4a16-a6ed-4167-91ad-0e600638ad9c
1,Koen H.,van Dam,10.1007/978-3-031-20179-0_3,6d82605a-4b5d-4c5a-9360-6dd8ee3a6051,van damk,Koen,van Dam,https://orcid.org/0000-0002-4879-9259,NaN,NaN,NaN,male,4c112571-59b8-4afc-b6cc-63905cc2fbe6
2,Michael,Vardon,10.1007/s13280-022-01757-5,17746a4b-c6d2-4920-8a35-cc5e393270ac,vardonm,Michael,Vardon,https://orcid.org/0000-0002-5708-6824,NaN,NaN,NaN,male,5749ceef-715d-48e8-a5c3-2cdbeca26a15
3,Paul,Lucas,10.1007/s13280-022-01757-5,17746a4b-c6d2-4920-8a35-cc5e393270ac,lucasp,Paul,Lucas,https://orcid.org/0000-0003-0292-7830,NaN,NaN,NaN,male,faa76dd9-c08a-426c-af48-582f0676d59c
4,Steve,Bass,10.1007/s13280-022-01757-5,17746a4b-c6d2-4920-8a35-cc5e393270ac,basss,Steve,Bass,NaN,https://scholar.google.co.uk/citations?hl=en&u...,NaN,NaN,male,52af1594-19d2-4e9b-bb3d-2c4f19283726
...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,John,Harrison,10.1016/j.dib.2022.108836,48fdaa55-7bbd-4d75-ba1f-231a30075f7b,harrisonj,John,Harrison,https://orcid.org/0000-0002-6434-5142,NaN,NaN,NaN,male,497e2199-283f-4b19-b709-481acbf5da8d
200,Mark,Howells,10.1016/j.dib.2022.108836,48fdaa55-7bbd-4d75-ba1f-231a30075f7b,howellsm,Mark,Howells,https://orcid.org/0000-0001-6419-4957,NaN,NaN,NaN,male,de7075f2-8cf5-459f-a754-34684a808d0d
201,Rudolf,Yeganyan,10.1016/j.dib.2022.108836,48fdaa55-7bbd-4d75-ba1f-231a30075f7b,yeganyanr,Rudolf,Yeganyan,https://orcid.org/0000-0001-9325-8147,NaN,NaN,NaN,male,33a47266-b0b8-4ca7-9fc8-7f1f3908e6f3
202,Shaikh M. S. U.,Eskander,10.3390/su14010453,b045aef4-b0aa-45a7-b9d4-1e9a399af475,eskanders,Shaikh M. S. U.,Eskander,NaN,NaN,NaN,NaN,male,2536def5-f55f-41bc-9934-5859cce812e3


In [73]:
authors_surname[authors_surname.index.str.contains('gebr')]

,First Name,Last Name,Orcid,google_scholar,pubmed,institution_url,gender,uuid
name,,,,,,,,
gebresilassiee,Eshetu Gebrekidan,Gebresilassie,https://orcid.org/0000-0001-8834-2766,NaN,NaN,NaN,male,470add76-d703-406e-ba15-d2af3ca99522
gebreslassiem,Mulualem,Gebreslassie,https://orcid.org/0000-0002-5509-5866,NaN,NaN,NaN,male,0f61f058-a41a-44ec-bfb5-c39aac18a165


In [74]:
relations[relations['join_name'].str.contains('gebr')]

,First_Name,Last_Name,DOI,paper_uuid,join_name
46,Eshetu Gebrekidan,Gebresilassie,10.1038/s41560-022-01152-0,bdcd53b3-034c-479d-8115-aaaf1d520f2c,gebresilassiee
47,Mulualem,Gebreslassie,10.1038/s41560-022-01152-0,bdcd53b3-034c-479d-8115-aaaf1d520f2c,gebreslassiem


In [75]:
joined.columns

Index(['First_Name', 'Last_Name', 'DOI', 'paper_uuid', 'join_name',
       'First Name', 'Last Name', 'Orcid', 'google_scholar', 'pubmed',
       'institution_url', 'gender', 'uuid'],
      dtype='object')

## Validate joined results

In [76]:
joined.columns

Index(['First_Name', 'Last_Name', 'DOI', 'paper_uuid', 'join_name',
       'First Name', 'Last Name', 'Orcid', 'google_scholar', 'pubmed',
       'institution_url', 'gender', 'uuid'],
      dtype='object')

In [77]:
joined[joined['First_Name'].str.strip().str.get(0) != joined['First Name'].str.strip().str.get(0)]

,First_Name,Last_Name,DOI,paper_uuid,join_name,First Name,Last Name,Orcid,google_scholar,pubmed,institution_url,gender,uuid


In [78]:
joined.to_excel('data/joined.xlsx')

In [79]:
joined.sample()

,First_Name,Last_Name,DOI,paper_uuid,join_name,First Name,Last Name,Orcid,google_scholar,pubmed,institution_url,gender,uuid
124,F.,Gardumi,10.1016/j.esd.2022.07.006,70aa944a-67f8-4c9c-8fe5-8248039a31b7,gardumif,Francesco,Gardumi,https://orcid.org/0000-0001-8371-9325,NaN,NaN,NaN,male,90cd58b0-23c4-41b7-bba3-470cbcb889b3


In [80]:
for column in ['First Name', 'Last Name', 'Orcid']:
    print(column)
    joined[column] = joined[column].str.strip()
authors = joined[['uuid', 'First Name', 'Last Name', 'Orcid', 'google_scholar', 'pubmed', 'institution_url', 'gender']]
authors = authors.drop_duplicates(keep='first')

First Name
Last Name
Orcid


In [81]:
authors.to_csv('data/authors.csv', index=False)

In [82]:
relations = joined[['uuid','paper_uuid']]
relations.to_csv('data/relations.csv', index=False)

papers.to_csv('data/papers.csv', index=False)

# Add genders

,uuid,First Name,Last Name,Orcid,google_scholar,pubmed,institution_url,gender
0,e851cf3d-1c2f-4a4d-a1a2-61e4275ea2bc,Liu,Yang,https://orcid.org/0000-0002-3363-8620,NaN,NaN,NaN,male
1,7765a41c-57f9-497b-9a31-7bd2537544f6,Koen,van Dam,https://orcid.org/0000-0002-4879-9259,NaN,NaN,NaN,male
2,fc7e83e7-4f64-4a73-be71-38edc6281f4b,Michael,Vardon,https://orcid.org/0000-0002-5708-6824,NaN,NaN,NaN,male
3,743b0d6b-f946-4ce9-903b-6885bd2c0e0f,Paul,Lucas,https://orcid.org/0000-0003-0292-7830,NaN,NaN,NaN,male
4,b640402a-46d6-48e7-9f99-d120404d717a,Steve,Bass,NaN,https://scholar.google.co.uk/citations?hl=en&u...,NaN,NaN,male
...,...,...,...,...,...,...,...,...
194,7d75d0fb-6303-444c-80a0-c023d0e56170,Layla,Khanfar,NaN,NaN,NaN,NaN,female
198,376b13a9-836d-44a7-856d-f67744da1ca4,Naomi,Tan,https://orcid.org/0000-0001-7957-8451,NaN,NaN,NaN,female
199,9b9e1d67-3fa9-43b2-b65e-3fbaed6e0e3c,John,Harrison,https://orcid.org/0000-0002-6434-5142,NaN,NaN,NaN,male
201,dd043f0d-9f53-480d-9edf-385c5899b7fc,Rudolf,Yeganyan,https://orcid.org/0000-0001-9325-8147,NaN,NaN,NaN,male


In [20]:
genders = pd.read_csv('data/authors_gendered.csv', usecols=['First Name', 'Last Name', 'Orcid', 'gender'])
genders['name'] = genders['Last Name'].str.strip().str.lower() + genders['First Name'].str.strip().str.lower().str.get(0)
authors = authors.join(genders[~genders['Orcid'].isna()].set_index('Orcid')['gender'], on='Orcid', how='right')
authors['name'] = authors['Last Name'].str.strip().str.lower() + authors['First Name'].str.strip().str.lower().str.get(0)
authors = authors.join(genders.set_index('name')['gender'], on='name', how='left', rsuffix='name_')

ValueError: columns overlap but no suffix specified: Index(['gender'], dtype='object')

In [30]:
authors

,uuid,First Name,Last Name,Orcid,google_scholar,pubmed,institution_url,gender
0,e851cf3d-1c2f-4a4d-a1a2-61e4275ea2bc,Liu,Yang,https://orcid.org/0000-0002-3363-8620,NaN,NaN,NaN,male
1,7765a41c-57f9-497b-9a31-7bd2537544f6,Koen,van Dam,https://orcid.org/0000-0002-4879-9259,NaN,NaN,NaN,male
2,fc7e83e7-4f64-4a73-be71-38edc6281f4b,Michael,Vardon,https://orcid.org/0000-0002-5708-6824,NaN,NaN,NaN,male
3,743b0d6b-f946-4ce9-903b-6885bd2c0e0f,Paul,Lucas,https://orcid.org/0000-0003-0292-7830,NaN,NaN,NaN,male
4,b640402a-46d6-48e7-9f99-d120404d717a,Steve,Bass,NaN,https://scholar.google.co.uk/citations?hl=en&u...,NaN,NaN,male
...,...,...,...,...,...,...,...,...
194,7d75d0fb-6303-444c-80a0-c023d0e56170,Layla,Khanfar,NaN,NaN,NaN,NaN,female
198,376b13a9-836d-44a7-856d-f67744da1ca4,Naomi,Tan,https://orcid.org/0000-0001-7957-8451,NaN,NaN,NaN,female
199,9b9e1d67-3fa9-43b2-b65e-3fbaed6e0e3c,John,Harrison,https://orcid.org/0000-0002-6434-5142,NaN,NaN,NaN,male
201,dd043f0d-9f53-480d-9edf-385c5899b7fc,Rudolf,Yeganyan,https://orcid.org/0000-0001-9325-8147,NaN,NaN,NaN,male


In [31]:
g.serialize('authors.ttl')

NameError: name 'g' is not defined

## Returns the abstracts of the papers of a given author

```sparql
PREFIX schema: <https://schema.org/>

SELECT ?abstract
WHERE {
    ?s a schema:ScholarlyArticle .
    ?s schema:abstract ?abstract .
    ?s schema:author ?author .
    ?author schema:name "Ioannis Pappis" .
} LIMIT 100
```

## Returns the number of publications by author

```sparql
PREFIX schema: <https://schema.org/>

SELECT ?name (COUNT(?s) as ?count)
WHERE {
    ?s a schema:ScholarlyArticle .
    ?s schema:author ?author .
    ?author schema:name ?name .
}
GROUP BY ?name
ORDER BY DESC(?count)
```

## Number of authors per open licensed publication

```sparql
PREFIX schema: <https://schema.org/>

SELECT ?title (COUNT(?author) as ?count)
WHERE {
    ?s a schema:ScholarlyArticle .
    ?s schema:title ?title .
    ?s schema:license ?license
    FILTER CONTAINS(?license, "licenses/by/4.0") .
    ?s schema:author ?author .
}
GROUP BY ?title
```

,uuid,First Name,Last Name,Orcid,google_scholar,pubmed,institution_url,gender
0,NaN,Liu,Yang,https://orcid.org/0000-0002-3363-8620,NaN,NaN,NaN,male
1,NaN,Koen,van Dam,https://orcid.org/0000-0002-4879-9259,NaN,NaN,NaN,male
2,NaN,Michael,Vardon,https://orcid.org/0000-0002-5708-6824,NaN,NaN,NaN,male
3,NaN,Paul,Lucas,https://orcid.org/0000-0003-0292-7830,NaN,NaN,NaN,male
4,NaN,Steve,Bass,NaN,https://scholar.google.co.uk/citations?hl=en&u...,NaN,NaN,male
...,...,...,...,...,...,...,...,...
194,NaN,Layla,Khanfar,NaN,NaN,NaN,NaN,female
198,NaN,Naomi,Tan,https://orcid.org/0000-0001-7957-8451,NaN,NaN,NaN,female
199,NaN,John,Harrison,https://orcid.org/0000-0002-6434-5142,NaN,NaN,NaN,male
201,NaN,Rudolf,Yeganyan,https://orcid.org/0000-0001-9325-8147,NaN,NaN,NaN,male
